In [4]:
import database_connected as db
import time
from selenium import webdriver
import re
import pandas as pd

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)
driver.get('http://prod.danawa.com/list/?cate=19227166')
# X월 먹거리 카테고리 http://prod.danawa.com/list/?cate=19227166
# 고양이 사료 카테고리 http://prod.danawa.com/list/?cate=19223561
# 강아지 사료 카테고리 http://prod.danawa.com/list/?cate=19238134

p_1_1 = re.compile('\W\d+k*g') # (100g or (100kg 찾기
p_1_2 = re.compile('x\d+\D') # x10개 찾기 
p_g = re.compile('\d+k*g') # g or kg 찾기
p_ml = re.compile('\d+ml') 
p_number = re.compile('\d+\.?\d*')
p_string = re.compile('\D*g')

def next_page():
    next_page = driver.find_element_by_css_selector('.now_on').text
    next_page = int(next_page) + 1
    try:
        driver.find_element_by_css_selector(f'[onclick^="javascript:movePage({next_page}"]').click()
        time.sleep(1)
    except Exception as e:
        try:
            driver.find_element_by_css_selector('.nav_next').click()
            time.sleep(1)
        except:
            return True
    return False


def db_collect(): 
    time.sleep(1)
    li_tags = driver.find_elements_by_css_selector("li[id^='productItem']")

    for i, tag in enumerate(li_tags):
        product_name = tag.find_element_by_css_selector("div > div.prod_info > p > a").text # 제품 이름 
        product_name = product_name.replace("'",'')
        product_desc = tag.find_element_by_css_selector("div[class='spec_list']").text # 제품 상세 설명 
        parent_pcode = tag.get_attribute('id') 
        #현재 태그에 존재하는 id 값을 가져와야 함
        parent_pcode = parent_pcode.replace('productItem','')
        open_button = tag.find_elements_by_css_selector("a[class='open']")
        
        parent_dict = { 
            'product_name': product_name,
            'product_desc': product_desc,
            'parent_pcode': parent_pcode,
        }
        db.push_prod_data_parent(data=parent_dict)




        if len(open_button) > 0:
            open_button[0].click()
        
        detail_list = tag.find_elements_by_css_selector("li[id^='productInfoDetail']")
        for j, detail_tag in enumerate(detail_list):
            weight = detail_tag.find_element_by_css_selector("p[class='memory_sect']").text # 제품 무게
            prod_name = product_name
            prod_name = prod_name.replace(" ", "")
            weight = weight.replace(" ","")
            weight = weight.replace("1위","")
            weight = weight.replace("2위","")
            weight = weight.replace(",","")
            weight = weight.replace("X","x")
            if "/" in weight :
                price_per_kg = detail_tag.find_element_by_css_selector("span[class='memory_price_sect']").text
                weight = weight.replace(price_per_kg, "")
            else:
                weight = weight 
            if '(' in weight : # weight 괄호 O 
                weight_1 = p_1_1.findall(weight)[0].replace("(","")
                weight_final = p_number.findall(weight_1)[0]
                weight_unit = p_string.findall(weight_1)[0]
                gift_unit = 1
                sell_unit = p_1_2.findall(weight)[0].replace(")","")
                sell_unit = p_number.findall(sell_unit)[0]

            else :
                if len(p_g.findall(weight)) > 0 : # weight 괄호 X, kg이나 g이 존재 
                    weight_final = p_number.findall(weight)[0]
                    weight_unit = p_string.findall(weight)[0]
                    gift_unit = 1
                    sell_unit = 1
                elif '개' in weight : # weight 괄호 X, 개가 존재 
                    if len(p_1_1.findall(prod_name)) > 0  : # 상품이름에 괄호 존재 
                        weight_1 = p_1_1.findall(prod_name)[0].replace("(","")
                        weight_final = p_number.findall(weight_1)[0]
                        weight_unit = p_string.findall(weight_1)[0]
                        gift_unit = p_1_2.findall(prod_name)[0].replace(")","")
                        gift_unit = p_number.findall(gift_unit)[0]
                        sell_unit = 1
                    elif 'ml' in prod_name : # 상품 이름에 괄호 없음 + ml 존재
                        weight_1 = p_ml.findall(prod_name)[0]
                        weight_final = p_number.findall(weight_1)[0]
                        weight_unit = "ml"
                        gift_unit = 1
                        sell_unit = p_number.findall(weight)[0]
                    elif len(p_g.findall(prod_name)) > 0 : # 상품 이름에 괄호 없음 + 숫자g or 숫자kg이 존재 
                        weight_1 = p_g.findall(prod_name)[0]
                        weight_final = p_number.findall(weight_1)[0]
                        weight_unit = p_string.findall(weight_1)[0]
                        gift_unit = 1
                        sell_unit = p_number.findall(weight)[0]
                    else : 
                        weight_final = p_number.findall(weight)[0]
                        weight_unit = '개'
                        gift_unit = 1
                        sell_unit = 1
            price = detail_tag.find_element_by_css_selector("p[class='price_sect'] > a > strong").text #제품 가격
            price = price.replace(',','')
            if '품절' in price or '가격비교예정' in price or '가격비교중지' in price:
                price = 0
                price_per_kg = ''
            try:
                price_per_kg = detail_tag.find_element_by_css_selector("span[class='memory_price_sect']").text #여기 이해 필요 memory_price_sect에 해당하는 text의 값을 price_per_kg에 담고, 해당하는 값에서 트림한것이 weight과 동일하다면 오류가 발생하는데 -> 이 때 price를 0으로 설정해준다
                weight = weight.replace(price_per_kg,'').strip()
            except:
                no_sale = detail_tag.find_element_by_css_selector("strong[class='no_sale']").text
                if '품절' in no_sale:
                    price = '0'

                    
            pcode = detail_tag.get_attribute('id')
            pcode = pcode.replace('productInfoDetail_','')
            weight_unit = weight_unit.replace('/n',"")
            detail_dict = { 
                'pcode': pcode,
                'parent_pcode': parent_pcode,
                'weight': weight_final,
                'weight_unit' : weight_unit,
                'gift_unit' : gift_unit,
                'sell_unit' : sell_unit,
                'price': price,
                'price_per_kg': price_per_kg
            }
            db.push_prod_data_detail(data=detail_dict)


In [10]:

go_stop = False
cnt = 1 #시작 페이지가 어딘지 
while not go_stop:
    print(cnt)
    db_collect()
    go_stop = next_page() 
    cnt += 1

1


KeyboardInterrupt: 

In [ ]:
driver.quit()

In [6]:
db_collect()

IndexError: list index out of range